In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
import os

In [ ]:
# first let's show the differences between our three classes
plt.figure(figsize = (15, 15))
covid_img   = cv2.imread('Covid19-dataset/train/Covid/01.jpeg')
normal_img  = cv2.imread('Covid19-dataset/train/Normal/01.jpeg')
vp_img      = cv2.imread('Covid19-dataset/train/Viral Pneumonia/01.jpeg')
plt.subplot(1, 3, 1)
plt.imshow(covid_img)
plt.title('Covid')
plt.subplot(1, 3, 2)
plt.imshow(normal_img)
plt.title('Normal')
plt.subplot(1, 3, 3)
plt.imshow(vp_img)
plt.title('Viral Pneumonia')
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import MobileNetV2, VGG16, VGG19, ResNet50
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# number of classes 
dir_list = sorted(os.listdir('Covid19-dataset/train'))
class_names_lst = [dir_name for dir_name in dir_list]
class_names_lst

In [ ]:
# let's load the data
img_generator = ImageDataGenerator(
    rescale = 1. / 255,
    horizontal_flip = True,
    rotation_range = 20,
    shear_range = 0.2,
    fill_mode = 'nearest',
    validation_split=0.1
)
train_data = img_generator.flow_from_directory(
    'Covid19-dataset/train',
    target_size = (255, 255),
    color_mode = 'rgb',
    classes = class_names_lst,
    class_mode = 'categorical',
    batch_size  = 64,
    subset = 'training'
)
valid_data = img_generator.flow_from_directory(
    'Covid19-dataset/train',
    target_size = (255, 255),
    color_mode = 'rgb',
    classes = class_names_lst,
    class_mode = 'categorical',
    batch_size  = 64,
    subset = 'validation'
)
test_generator = ImageDataGenerator(
    rescale = 1. / 255
)
test_data = test_generator.flow_from_directory(
    'Covid19-dataset/test',
    target_size = (255, 255),
    color_mode = 'rgb',
    classes = class_names_lst,
    class_mode = 'categorical',
    batch_size  = 64,
    shuffle = False
)

In [ ]:
base_model = ResNet50(input_shape=(255, 255, 3),
                      include_top=False,
                      weights="imagenet")
base2_model = MobileNetV2(input_shape=(255, 255, 3),
                          include_top=False,
                          weights="imagenet")
base3_model = VGG19(input_shape=(255, 255, 3),
                    include_top=False,
                    weights="imagenet")
base_model.trainable = False
base2_model.trainable = False
base3_model.trainable = True

In [ ]:
len(base3_model.layers)

In [ ]:
fine_tune = 20
for layer in base_model.layers[:fine_tune]:
    layer.trainable = False

In [ ]:
early_stop=EarlyStopping(monitor='val_loss',patience=5, restore_best_weights=True)

In [ ]:
model = keras.Sequential([
    base3_model,
    layers.Flatten(),
    layers.Dense(4, activation = 'softmax')
])
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001),
    loss = 'categorical_crossentropy',
    metrics = 'accuracy'
)
model.summary()

In [ ]:
model.fit(
    train_data,
    epochs = 2,
    batch_size = 64,
    validation_data = valid_data,
    callbacks = [early_stop]
)

In [ ]:
y_hat = model.predict(test_data)

In [ ]:
y_hat_lst = []
for idx in range(y_hat.shape[0]):
    y_hat_lst.append(np.argmax(y_hat[idx]))

In [ ]:
y_hat_cat = []
for val in y_hat_lst:
    y_hat_cat.append(class_names_lst[val])

In [ ]:
true_y_cat = []
for val in test_data.classes:
    true_y_cat.append(class_names_lst[val])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(test_data.classes, y_hat_lst)

In [ ]:
for i, j in zip(true_y_cat, y_hat_cat):
    print(i, '|', j)

In [ ]:
# matplotlib chart plot showing number of correct and incorrect predictions of normal x-rays 
number_of_correct_normal = 0
number_of_incorrect_normal = 0
number_of_correct_covid = 0
number_of_incorrect_covid = 0
number_of_correct_vp = 0
number_of_incorrect_vp = 0
for i, j in zip(true_y_cat, y_hat_cat):
    if i == j:
        if i == 'Normal':
            number_of_correct_normal += 1
        elif i == 'Covid':
            number_of_correct_covid += 1
        else:
            number_of_correct_vp += 1
    else:
        if i == 'Normal':
            number_of_incorrect_normal += 1
        elif i == 'Covid':
            number_of_incorrect_covid += 1
        else:
            number_of_incorrect_vp += 1

plt.figure(figsize = (15, 15))

# matplotlib chart plot showing number of correct and incorrect predictions of covid x-rays
plt.subplot(1, 3, 2)
plt.bar('Correct', number_of_correct_covid, color = 'green')
plt.bar('Incorrect', number_of_incorrect_covid, color = 'red')
plt.title('Covid X-Ray Predictions')
plt.show()

plt.subplot(1, 3, 1)
plt.bar('Correct', number_of_correct_normal, color = 'green')
plt.bar('Incorrect', number_of_incorrect_normal, color = 'red')
plt.title('Normal X-Ray Predictions')
plt.show()

# matplotlib chart plot showing number of correct and incorrect predictions of viral pneumonia x-rays
plt.subplot(1, 3, 3)
plt.bar('Correct', number_of_correct_vp, color = 'green')
plt.bar('Incorrect', number_of_incorrect_vp, color = 'red')
plt.title('Viral Pneumonia X-Ray Predictions')
plt.show()